# Import libraries and Data

In [1]:
import pandas as pd
import numpy as np
import nltk
import re

In [2]:
df = pd.read_csv("/content/train.tsv",sep="\t",header=0)

In [3]:
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [4]:
train  =df[['Phrase', 'Sentiment']]

In [5]:
train.head()

,Phrase,Sentiment
0,A series of escapades demonstrating the adage ...,1
1,A series of escapades demonstrating the adage ...,2
2,A series,2
3,A,2
4,series,2


In [6]:
df_test = pd.read_csv("/content/test.tsv", sep='\t', header=0)

# Preprocessing of Data

In [8]:
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

for index, row in train.iterrows():
    filter_sentence = ''
    sentence = row['Phrase']
    
    # Cleaning the sentence with regex
    sentence = re.sub(r'[^\w\s]', '', sentence)
    
    # Tokenization
    words = nltk.word_tokenize(sentence)

    # Stopwords removal
    words = [w for w in words if not w in stop_words]

    for words in words:
        filter_sentence = filter_sentence  + ' ' + str(lemmatizer.lemmatize(words)).lower()

    train.loc[index, 'Phrase'] = filter_sentence

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [11]:
for index, row in df_test.iterrows():
    filter_sentence = ''
    sentence = row['Phrase']
    
    # Cleaning the sentence with regex
    sentence = re.sub(r'[^\w\s]', '', sentence)
    
    # Tokenization
    words = nltk.word_tokenize(sentence)

    # Stopwords removal
    words = [w for w in words if not w in stop_words]

    for words in words:
        filter_sentence = filter_sentence  + ' ' + str(lemmatizer.lemmatize(words)).lower()

    df_test.loc[index, 'Phrase'] = filter_sentence

In [12]:
train.head()

,Phrase,Sentiment
0,a series escapade demonstrating adage good go...,1
1,a series escapade demonstrating adage good goose,2
2,a series,2
3,a,2
4,series,2


In [13]:
df_test.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,an intermittently pleasing mostly routine effort
1,156062,8545,an intermittently pleasing mostly routine effort
2,156063,8545,an
3,156064,8545,intermittently pleasing mostly routine effort
4,156065,8545,intermittently pleasing mostly routine


In [14]:
test = df_test.Phrase

In [15]:
X_train = train.Phrase

In [16]:
Y_train = train.Sentiment

# Pipeline

In [17]:
from sklearn.pipeline import Pipeline
from sklearn import tree
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer(norm='l2')),
    ('clf', tree.DecisionTreeClassifier()),
])

In [19]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features=None, max_leaf_nodes=None,
              

In [20]:
pred_label = pipeline.predict(test)

In [21]:
pred_label.shape

(66292,)

In [23]:
pred_label

array([1, 1, 2, ..., 1, 1, 1])

In [24]:
answer = np.savetxt('ans.csv', pred_label,delimiter=',', )